# Analysis of Movie Critics, Their Opinions, and What Makes a Good Movie?

**Report Author:** Angad Kalra  
**SN:** 1005134999  
**Summary:** Analysis on NYT movie critics, movies they've reviewed, and characteristics of movies affecting revenue.

## Introduction

#### The purpose of this report is to answer three questions:
1. How accurate are current NYT movie critics when it comes to predicting box office hits?
2. Do the critic's movie reviews align with whether the movie was a critic's pick?
3. What characteristics are unique among movies that perform well financially?

#### Due to limited API requests, only recent movies and reviews were considered. 

## Methods

Q1:  
* For each critic, their most recent picks were gathered from NYT API; for each pick, the box office total was requested from TMDB API. 
* To determine the accuracy of a critic, I counted how many critic picks had box office totals greater than $100 million and divided that number by the total number of picks.  

Q2:  
* For each critic, sentiment analysis was performed on their recent movie reviews (scraped from NYT website) resulting in a value between -1 and 1 (sentiment polarity).  
* Each review was plotted on a 2 dimensional plot with their x-coordinate being sentiment polarity and y-coordinate being 1 or 0, depending on if the movie was a critic's pick or not. 
* (still need to determine relationship...)  

Q3:  
* Using the TMDB API, I retrieved all movies released between January 2010 and January 2017 and sorted them by descending revenue.  
* Movies ranked 1st - 100th were put into one dataset, and movies ranked 200th - 300th were put into another dataset.
* Movie characteristics compared were budget, genres, and release season.  
* To compare budget, a side-by-side boxplot was used.  
* To compare release season, mode was used.  
* To compare genres, I will count the frequency of each genre in each dataset and rank them in descending order.  

## Results

Q1:  
* Critic: A. O. Scott, # Picks: 20, Accuracy: 0.35

* Critic: Manohla Dargis, # Picks: 27, Accuracy: 0.41

* Critic: Stephen Holden, # Picks: 11, Accuracy: 0.0  

Q2:  
* (going to insert 2D plot here with appropriate axis and labels...need to ask Prof for assistance.)

Q3:  
* (going to insert boxplot for budget)
* Mode Release Season:
    * Top 100 Movies: Summer
    * Bottom 100 Movies: Summer
* (going to insert genre rankings here...)

## Conclusion

Q1:  
* Result Summary: 
    * All three critics had accuracies well below 50%, with the highest being 41% and lowest being 0%.
* Limitations:
    * The API from which revenue was retrieved had a limit on the number of requests per 10 seconds. 
    * The box office hit benchmark was arbitrary and chosen by me. 
* Future Considerations:
    * Using more sophisticated API requests, retrieve more movie reviews per critic to get better accuracy measure. 
    * Analyze historical box office data to get more accurate benchmark of a box office hit. 

Q2:  
* Result Summary: 
    * TODO...
* Limitations:
    * TODO... 
* Future Considerations:
    * Use different sentiment analysis packages and compare results.
    * TODO...


Q3:  
* Result Summary: 
    * TODO...
* Limitations:
    * The API from which revenue was retrieved had a limit on the number of requests per 10 seconds. 
    * TODO... 
* Future Considerations:
    * Using more sophisticated API requests, retrieve more movies for both datasets. 
    * Include other influential movie characteristics such as director, main actors, movie language, part of sequel/trilogy/series, production companies.  
    * TODO...